In [2]:
import os
ROOT = "/content/drive/MyDrive/ineuron/DL"
os.chdir(ROOT)

In [3]:
assert os.getcwd() == ROOT

In [4]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11490434/11490434 [==============================] - 2s 0us/step


In [6]:
tf.random.set_seed(42)
np.random.seed(42)

In [7]:
LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 13s - loss: 1.5464 - accuracy: 0.6019 - val_loss: 0.9486 - val_accuracy: 0.8002 - 13s/epoch - 8ms/step
Epoch 2/10
1719/1719 - 7s - loss: 0.7480 - accuracy: 0.8265 - val_loss: 0.5900 - val_accuracy: 0.8578 - 7s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 7s - loss: 0.5410 - accuracy: 0.8621 - val_loss: 0.4712 - val_accuracy: 0.8786 - 7s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.4587 - accuracy: 0.8773 - val_loss: 0.4134 - val_accuracy: 0.8922 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 6s - loss: 0.4144 - accuracy: 0.8863 - val_loss: 0.3785 - val_accuracy: 0.8986 - 6s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 6s - loss: 0.3860 - accuracy: 0.8919 - val_loss: 0.3554 - val_accuracy: 0.9040 - 6s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 6s - loss: 0.3658 - accuracy: 0.8966 - val_loss: 0.3381 - val_accuracy: 0.9082 - 6s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.3502 - accuracy: 0.9004 - val_loss: 0.3245 - val_accuracy: 0.9118 - 4s/epoch - 3ms/step
Epoch 

In [11]:
model.save("pretrained_mnist_model.h5")

# Transfer Learning:

## New Problem Statement-
Classify handwritten digits into odd and even

In [12]:
# load the previous model
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [13]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [14]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [15]:
for layer in pretrained_mnist_model.layers[:-1]: # leaves the last layer unfreezed => trainable
  layer.trainable = False # freezing the layers from getting trained
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [16]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [17]:
try:
  del new_model
except:
  pass

In [18]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [19]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

200 * 2(weights) + 2 (bias)

In [20]:
# remove last layer
266610 - 1010

265600

In [21]:
265600 + 202

265802

In [22]:
# less parameters to train
# less number of time to train
# less resources and thus less cost